In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from create_model import create_model

gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

abp = pd.read_csv('data/abp_data.csv', index_col='index')
abp = abp.iloc[1:58652]
abp.drop(labels='sample_id', axis=1, inplace=True)
print(len(abp))

pleth = pd.read_csv('data/pleth_data.csv', index_col='index')
pleth.drop(index=[0, 58652], inplace=True)
pleth.drop(labels='sample_id', axis=1, inplace=True)
print(len(pleth))

labels = np.array([np.array([abp.iloc[i]['sbp'], abp.iloc[i]['dbp']]) for i in range(len(abp))])
print(labels.shape)

windows = np.array([np.array(pleth.iloc[i]).reshape((625, 1)) for i in range(len(pleth))])
print(windows.shape)

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((windows, labels))
dataset = dataset.shuffle(buffer_size=1000, seed=69420).batch(32).cache().prefetch(buffer_size=tf.data.AUTOTUNE)

train_ds_size = int(0.70 * len(labels))
val_ds_size = int(0.15 * len(labels))

train_ds = dataset.take(train_ds_size)
remaining = dataset.skip(train_ds_size)  
val_ds = remaining.take(val_ds_size)
test_ds = remaining.skip(val_ds_size)

In [ ]:
model = create_model()
model.fit(train_ds,
          validation_data=val_ds,
          epochs=5)